# 文本相似度实例

文本匹配其实是一个较为宽泛的概念，基本上只要涉及两段文本之间关系的，都可以被看做是一种文本匹配的任务，只是在具体的场景下不同的任务对匹配的定义可能是存在差异的，具体的任务包括文本相似度计算、问答和对话匹配、文本推理等等，甚至抽取式机器阅读理解和多项选择本质上也是文本匹配。只不过这里专门针对文本相似度任务，判断两段文本是否表达了相同的语义，本质上也是一个二分类任务

这里只做单塔的交互模式，也就是两段文本用[SEP]拼接后输入模型输出0~1之间的相似度，相比于将其视为相似/不相似的二分类问题，能直接从多个候选文本找到最相似的，但是在大规模情况下与全量数据都进行一遍推理计算相似度肯定不符合时间要求！

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

## Step1 导入相关包

In [2]:
import torch
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments # 又用回AutoModelForSequenceClassification了

/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [3]:
dataset = load_dataset("json", data_files="./train_pair_1w.json", split="train")
dataset

Dataset({
    features: ['sentence1', 'sentence2', 'label'],
    num_rows: 10000
})

In [4]:
dataset[0]

{'sentence1': '找一部小时候的动画片', 'sentence2': '求一部小时候的动画片。谢了', 'label': '1'}

## Step3 划分数据集

In [5]:
# load_dataset加载的数据可以直接用train_test_split划分训练集和测试集，测试集就拿来当验证集用，因为真正意义上的测试集应该是不带标签的
datasets = dataset.train_test_split(test_size=0.2) 
datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
})

## Step4 数据集预处理

In [6]:
tokenizer = AutoTokenizer.from_pretrained("/data/PLM/chinese-macbert-base")
tokenizer

BertTokenizerFast(name_or_path='/data/PLM/chinese-macbert-base', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [7]:
def process_function(examples):
    # 还是老样子，用text和text_pair，中间[SEP]隔开，padding到设定的最大长度还是每个batch的最大长度应该影响不大
    tokenized_examples = tokenizer(examples["sentence1"], examples["sentence2"], max_length=128, truncation=True) 
    # labels本来是一个个字符串，现在改成float以输入，本来其实int应该也可以，但这里我们希望模型只有一个输出可以用mse loss所以用float
    tokenized_examples["labels"] = [float(label) for label in examples["label"]] 
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

Map: 100%|██████████| 2000/2000 [00:00<00:00, 6847.31 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

In [8]:
print(tokenized_datasets["train"][0])

{'input_ids': [101, 3449, 7027, 7504, 4449, 1922, 1922, 1469, 3449, 6564, 2209, 4385, 1762, 2582, 3416, 749, 8043, 102, 3031, 3031, 5074, 4638, 2797, 823, 858, 2207, 2140, 6564, 1036, 8013, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': 0.0}


## Step5 创建模型

In [9]:
from transformers import BertForSequenceClassification # 也可以用这个
# 注意num_labels=1会自动被认为是回归任务用MSELoss()；num_labels默认为2即二分类，用CrossEntropyLoss()
model = AutoModelForSequenceClassification.from_pretrained("/data/PLM/chinese-macbert-base", num_labels=1) 
# 多标签任务会自动使用BCEWithLogitsLoss()
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /data/PLM/chinese-macbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## Step6 创建评估函数

In [10]:
acc_metric = evaluate.load("/data/daiyw/Compare/evaluate/metrics/accuracy")
f1_metric = evaluate.load("/data/daiyw/Compare/evaluate/metrics/f1")

In [11]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = [int(p > 0.5) for p in predictions]
    # predictions = predictions.argmax(axis=-1) # argmax还可以这么做，叹为观止但又意料之中
    labels = [int(l) for l in labels]
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

## Step7 创建TrainingArguments

In [12]:
train_args = TrainingArguments(output_dir="./cross_model",      # 输出文件夹
                               per_device_train_batch_size=32,  # 训练时的batch_size
                               per_device_eval_batch_size=32,  # 验证时的batch_size
                               logging_steps=10,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True)     # 训练完成后加载最优模型
train_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_always_push=False,
hub_mod

## Step8 创建Trainer

In [13]:
from transformers import DataCollatorWithPadding
trainer = Trainer(model=model, 
                  args=train_args, 
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer), # 可以这么认为，留着tokenizer是为了padding，不然连pad_token_id都找不着
                  compute_metrics=eval_metric)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


## Step9 模型训练

In [14]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.116600,0.080824,0.901000,0.880723
2,0.089300,0.069455,0.912000,0.888183
3,0.054800,0.071615,0.913000,0.886571


TrainOutput(global_step=750, training_loss=0.08878439418474833, metrics={'train_runtime': 190.5341, 'train_samples_per_second': 125.962, 'train_steps_per_second': 3.936, 'total_flos': 1557471908166144.0, 'train_loss': 0.08878439418474833, 'epoch': 3.0})

## Step10 模型评估

In [15]:
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.0694550946354866,
 'eval_accuracy': 0.912,
 'eval_f1': 0.8881829733163914,
 'eval_runtime': 4.7011,
 'eval_samples_per_second': 425.432,
 'eval_steps_per_second': 13.401,
 'epoch': 3.0}

## Step11 模型预测

In [16]:
from transformers import pipeline, TextClassificationPipeline # 应该也可以直接用TextClassificationPipeline，无需设置"text-classification"

In [17]:
model.config.id2label = {0: "不相似", 1: "相似"}

In [18]:
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [19]:
# 如果不加function_to_apply="none"，会使用默认值"default"，也就是模型预定义的标准处理方法
# 因为pipeline没有对sigmoid做很好地处理，所以会得到极其错误的结果
result = pipe({"text": "我喜欢北京", "text_pair": "天气怎样"}, function_to_apply="none") # 指定两个参数，以字典的形式传进去！
# 就算加了function_to_apply="none"，也要根据输出的score才能得到最终的判断
result["label"] = "相似" if result["score"] > 0.5 else "不相似" # 所以这个score就是我们回归出来的直接结果
result

{'label': '不相似', 'score': 0.019719230011105537}